In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download data

In [2]:
ejesviales = gpd.read_file('../../data/external/temporal_todocker/0_marcogeo_ejesviales_prueba.gpkg')
hubdistance = gpd.read_file('../../data/external/temporal_todocker/02_hubdistance.gpkg')

In [85]:
print(ejesviales.shape)
ejesviales.head(1)

(768, 13)


,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,geometry
0,140390001,14,039,0001,00184,00003,Teatro de la República,Calle,Un sentido,1,Urbana,1403900010018400003,"MULTILINESTRING ((666274.061 2287716.662, 6662..."


In [86]:
print(hubdistance.shape)
hubdistance.head(2)

(23044, 15)


,fid,cat,Tipo_de_se,Clasificac,Clasifica0,idruta,fid_2,cat_2,Tipo_de_se_2,Clasificac_2,Clasifica0_2,idruta_2,HubName,HubDist,geometry
0,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,2,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,1412000010358500034,2.604311,POINT (665756.141 2287120.589)
1,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,165,2,Complementaria,Rutas complementarias y otros servicios,Operando,36,1412000010358500034,2.604311,POINT (665756.141 2287120.589)


In [87]:
columns_tokeep = ['idruta','HubName','geometry']
hubdistance_f = hubdistance[columns_tokeep]
hubdistance_f = hubdistance_f.rename(columns={'HubName':'segmentocalle'})
hubdistance_f.head(1)

,idruta,segmentocalle,geometry
0,34,1412000010358500034,POINT (665756.141 2287120.589)


In [88]:
rutas = list(hubdistance_f.idruta.unique())
print(len(rutas))

segmentos = list(hubdistance_f.segmentocalle.unique())
print(len(segmentos))

26
138


## Iteración (Método anterior, no permite ajustar el número de veces que se debe encontrar una ruta en un segmento para considerar que pasa por ahí y no solo es una intersección)

In [89]:
#hubdistance_f.head(2)

In [90]:
# ---------- Diccionario de segmentos que tienen rutas y las rutas que tienen
#dicc_segmentos_rutas = {}
#listavacia = []

# ---------- Crear un diccionario con los segmentos de calle y una lista de rutas por segmento
#for index,row in hubdistance_f.iterrows():    
#    segmento_calle = row['segmentocalle'] #Es un str   
#    ruta = row['idruta'] #Es un int
    
    # ---------- Si es la primera vez que encuentra este segmento, agregarlo con una lista vacía
#    if segmento_calle not in dicc_segmentos_rutas:
#        dicc_segmentos_rutas[segmento_calle] = listavacia.copy()

    # ---------- Para esta row en la que iteramos (segmento de calle y ruta), si es la primera vez que encuentra la ruta en este segmento, agregarla a esa lista:
#    if ruta not in dicc_segmentos_rutas[segmento_calle]:
#        dicc_segmentos_rutas[segmento_calle].append(ruta)

## Data processing 01 - Diccionario con segmentos y rutas que pasan por cada segmento

In [91]:
#Rutas en el gdf
rutas = list(hubdistance_f.idruta.unique())

In [130]:
#---------- Insumos base de la iteración ----------
dicc_segmentos_rutas = {} #Guarda los segmentos de calle y las rutas que pasan por cada segmento de calle
listavacia = [] #Lista vacía usada cuando se encuentra un nuevo segmento de calle
mincount = 9 #Número mínimo de ocasiones en las que se debe encontrar una ruta cerca de un segmento de calle para considerar que la ruta pasa por ahí y no solo que la encontró porque es una intersección
    
#---------- Iterar ruta por ruta ----------
for ruta in rutas:
    gdf_ruta = hubdistance_f[hubdistance_f.idruta == ruta]

    #---------- Veces que la ruta pasa por un segmento de calle ----------
    segs_deruta = gdf_ruta.groupby('segmentocalle').count()
    #---------- Formato ----------
    columns_tokeep = ['idruta']
    segs_deruta = segs_deruta[columns_tokeep]
    segs_deruta = segs_deruta.reset_index()
    segs_deruta = segs_deruta.rename(columns={'idruta':'count'})

    #---------- Para el count de segmentos esa ruta ----------
    for row_id,row_series in segs_deruta.iterrows():
        segmento_calle = row_series[0]
        count = row_series[1]
    
        #---------- Si se confirma que la ruta pasa por ahí, agregar al diccionario el segmento de calle y la ruta ----------
        if count > (mincount-1): 
            #---------- Si no se había encontrado ese segmento de calle, meterlo al diccionario por primera vez con una lista vacia de rutas ----------
            if segmento_calle not in dicc_segmentos_rutas:
                dicc_segmentos_rutas[segmento_calle] = listavacia.copy()
            
            #---------- Si la ruta no está ya asignada a ese segmento, meterla a la lista ----------
            if ruta not in dicc_segmentos_rutas[segmento_calle]:
                dicc_segmentos_rutas[segmento_calle].append(ruta)

In [131]:
#Ejemplo:
dicc_segmentos_rutas['1403900010015900003']

[42, 43, 44, 45, 46, 47]

## Data processing 02 - Asignar numero de rutas a los segmentos de ejesviales

In [132]:
# Crear un gdf de ejes viales con rutas_count
ejesviales_rutas = ejesviales.copy()
ejesviales_rutas['rutas_count'] = 0

In [133]:
ejesviales_rutas.head(1)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,geometry,rutas_count
0,140390001,14,039,0001,00184,00003,Teatro de la República,Calle,Un sentido,1,Urbana,1403900010018400003,"MULTILINESTRING ((666274.061 2287716.662, 6662...",0


In [140]:
# Iterar en el diccionario creado para asignar el numero de rutas que pasan por cada segmento
for segmento in dicc_segmentos_rutas:
    idx = ejesviales_rutas['CVEGEOSEG'] == segmento
    ejesviales_rutas.loc[idx,'rutas_count'] = len(dicc_segmentos_rutas[segmento])

In [147]:
# Revisión rápida: En esta df (prueba) máximo deberían haber 14 rutas en una calle
ejesviales_rutas = ejesviales_rutas.sort_values(by = 'rutas_count', ascending = False)
ejesviales_rutas.head(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,geometry,rutas_count
366,141200001,14,120,0001,03585,00036,Vallarta Eje Poniente,Avenida,Dos sentidos,2,Urbana,1412000010358500036,"MULTILINESTRING ((666343.170 2286995.168, 6660...",14
364,141200001,14,120,0001,03585,00034,Vallarta Eje Poniente,Avenida,Dos sentidos,2,Urbana,1412000010358500034,"MULTILINESTRING ((666019.182 2287063.738, 6659...",14


## Save to db

In [136]:
aup.gdf_to_db_slow(ejesviales_rutas,"rutastransporte_bystreet", 'segundopiso', if_exists='replace')